In [1]:
import numpy as np
import pandas as pd
import re
from typing import List

In [2]:
class TFIDF:
    def __init__(self):
        # Từ điển lưu trữ IDF
        self.idf_dict = {}
        # Từ vựng
        self.vocabulary = []
    
    def remove_tags(self, text: str) -> np.ndarray:
        """
        Tiền xử lý văn bản
        
        Parameters:
        text (str): Văn bản đầu vào
        
        Returns:
        numpy.ndarray: Mảng các từ đã xử lý
        """
        result = re.sub(r'<.*?>','',text)           #xoá HTML tags, tiền tố r nghĩa là raw
        result = re.sub('https://.*','',result)     #xoá URLs
        result = re.sub(r'[^\w\s]', '',result)      #xoá non-alphanumeric characters ví dụ như các dấu câu
        result = result.lower()                     # Chuyển về chữ thường
        
        # Tách từ và chuyển thành numpy array
        return np.array(result.split())
    
    def fit(self, documents: List[str]) -> 'TFIDF':
        """
        Học từ vựng và tính IDF
        
        Parameters:
        documents (List[str]): Danh sách các văn bản
        
        Returns:
        self: Đối tượng hiện tại
        """
        # Xử lý văn bản thành numpy arrays
        processed_docs = [self.remove_tags(doc) for doc in documents]
        
        # Chuyển sang DataFrame để dễ xử lý
        df_docs = pd.DataFrame({'words': processed_docs})
        
        # Tổng số văn bản
        total_docs = len(documents)
        
        # Đếm số văn bản chứa từng từ
        word_doc_count = {}
        for doc_words in processed_docs:
            unique_words = set(doc_words)  # Chỉ đếm mỗi từ một lần trong một văn bản
            for word in unique_words:
                word_doc_count[word] = word_doc_count.get(word, 0) + 1
        
        # Tính IDF sử dụng numpy
        self.idf_dict = {
            word: np.log(total_docs / (count)) + 1 
            for word, count in word_doc_count.items()
        }
        
        # Lưu từ vựng
        self.vocabulary = list(word_doc_count.keys())
        
        return self
    
    def transform(self, documents: List[str]) -> np.ndarray:
        """
        Chuyển đổi văn bản thành ma trận TF-IDF
        
        Parameters:
        documents (List[str]): Danh sách các văn bản
        
        Returns:
        numpy.ndarray: Ma trận TF-IDF
        """
        # Khởi tạo ma trận kết quả
        tfidf_matrix = np.zeros((len(documents), len(self.vocabulary)))
        
        # Xử lý từng văn bản
        for i, doc in enumerate(documents):
            # Tiền xử lý văn bản
            words = self.remove_tags(doc)
            
            # Tính Term Frequency (TF) sử dụng pandas
            word_counts = pd.Series(words).value_counts()
            total_words = len(words)
            
            # Tạo vector cho văn bản
            for j, vocab_word in enumerate(self.vocabulary):
                # Tính TF
                tf = word_counts.get(vocab_word, 0) / total_words
                
                # Tính TF-IDF
                idf = self.idf_dict.get(vocab_word, 0)
                tfidf_matrix[i, j] = tf * idf
        
        return tfidf_matrix

In [3]:
    df = pd.read_csv('C:/Users/ADMIN/Movie/IMDB Dataset.csv')
    df.head()['review']

0    One of the other reviewers has mentioned that ...
1    A wonderful little production. <br /><br />The...
2    I thought this was a wonderful way to spend ti...
3    Basically there's a family where a little boy ...
4    Petter Mattei's "Love in the Time of Money" is...
Name: review, dtype: object

In [4]:
def main():
    df = pd.read_csv('C:/Users/ADMIN/Movie/IMDB Dataset.csv')
    dfs=df.head()
    reviews = dfs['review'].values
    reviews = reviews.tolist()
    reviews
    vectorizer = TFIDF()
    # Học từ vựng và IDF
    vectorizer.fit(reviews)
    
    # # In ra từ vựng
    # print("Từ vựng:")
    # print(vectorizer.vocabulary)

    # In ra IDF của các từ
    print("\nIDF của các từ:")
    for word, idf in vectorizer.idf_dict.items():
        print(f"{word}: {idf:.4f}")

    # Chuyển đổi văn bản thành ma trận TF-IDF
    tfidf_matrix = vectorizer.transform(reviews)
    
    # Tạo DataFrame để hiển thị kết quả
    tfidf_df = pd.DataFrame(
        tfidf_matrix, 
        columns=vectorizer.vocabulary
    )
    
    print("\nMa trận TF-IDF:")
    print(tfidf_df)

In [5]:
if __name__ == "__main__":
    main()


IDF của các từ:
one: 1.2231
high: 2.6094
oswald: 2.6094
privacy: 2.6094
security: 2.6094
gangstas: 2.6094
in: 1.2231
focuses: 2.6094
muslims: 2.6094
guards: 2.6094
on: 1.2231
1: 2.6094
happened: 2.6094
a: 1.0000
i: 1.5108
if: 1.9163
doesnt: 2.6094
appeal: 2.6094
drugs: 2.6094
where: 1.5108
inwards: 2.6094
uncomfortable: 2.6094
wholl: 2.6094
irish: 2.6094
watched: 1.9163
to: 1.0000
touch: 2.6094
darker: 2.6094
use: 1.9163
would: 2.6094
out: 1.5108
oz: 2.6094
word: 2.6094
be: 1.5108
graphic: 2.6094
called: 2.6094
experimental: 2.6094
fronts: 2.6094
show: 2.6094
em: 2.6094
set: 2.6094
bitches: 2.6094
viewingthats: 2.6094
of: 1.0000
home: 1.9163
glass: 2.6094
city: 1.9163
shows: 2.6094
ever: 2.6094
accustomed: 2.6094
hearted: 2.6094
no: 1.9163
section: 2.6094
this: 1.5108
your: 2.6094
away: 2.6094
and: 1.0000
goes: 2.6094
nickname: 2.6094
scuffles: 2.6094
after: 2.6094
mainstream: 2.6094
developed: 2.6094
around: 2.6094
was: 1.9163
state: 2.6094
order: 2.6094
charm: 2.6094
me: 2.6094
pull